In [5]:
import numpy as np

# Define grid world environment with partial observation
class GridWorldPOMDP:
    def __init__(self, width, height, goal, obstacles):
        self.width = width
        self.height = height
        self.goal = goal
        self.obstacles = obstacles
        self.agent_pos = (0, 0)

    def reset(self):
        self.agent_pos = (0, 0)
        return self.get_observation()

    def step(self, action):
        if isinstance(action, int):  # Check if action is an integer
            # Map action index to (dx, dy) tuple
            if action == 0:
                dx, dy = 0, 1  # Move up
            elif action == 1:
                dx, dy = 0, -1  # Move down
            elif action == 2:
                dx, dy = -1, 0  # Move left
            elif action == 3:
                dx, dy = 1, 0  # Move right
        else:  # Assume action is already a tuple
            dx, dy = action

        x, y = self.agent_pos
        new_x = min(max(x + dx, 0), self.width - 1)
        new_y = min(max(y + dy, 0), self.height - 1)

        # Print out the movement of the agent
        print(f"Agent moved from ({x}, {y}) to ({new_x}, {new_y})")

        self.agent_pos = (new_x, new_y)

        # Print out the grid world environment
        for i in range(self.height):
            for j in range(self.width):
                if (j, i) == self.agent_pos:
                    print(" A ", end="")
                elif (j, i) == self.goal:
                    print(" G ", end="")
                elif (j, i) in self.obstacles:
                    print(" X ", end="")
                else:
                    print(" . ", end="")
            print()  # Move to the next line for the next row

        reward = -1  # Small negative reward for each step
        done = self.agent_pos == self.goal  # Episode ends when goal is reached
        return self.get_observation(), reward, done


    def get_observation(self):
        # Partial observation: agent's position and nearby obstacle positions
        observed_obstacles = []
        x, y = self.agent_pos
        for i in range(x - 1, x + 2):
            for j in range(y - 1, y + 2):
                if (i, j) in self.obstacles:
                    observed_obstacles.append((i, j))
        return (self.agent_pos, tuple(observed_obstacles))

# Define Q-learning agent for partially observable state
class QLearningAgentPOMDP:
    def __init__(self, num_actions, learning_rate=0.1, discount_factor=0.9, epsilon=0.1):
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.Q = {}

    def get_Q_value(self, state, action):
        if (state, action) in self.Q:
            return self.Q[(state, action)]
        else:
            return 0  # Initialize Q-value to 0 for unseen state-action pairs

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            action_index = np.random.randint(self.num_actions)  # Exploration
        else:
            Q_values = [self.get_Q_value(state, a) for a in range(self.num_actions)]
            action_index = np.argmax(Q_values)  # Exploitation
        
        # Map action index to (dx, dy) tuple
        if action_index == 0:
            dx, dy = 0, 1  # Move up
        elif action_index == 1:
            dx, dy = 0, -1  # Move down
        elif action_index == 2:
            dx, dy = -1, 0  # Move left
        else:
            dx, dy = 1, 0  # Move right
        
        return (dx, dy)

    def update_Q_value(self, state, action, reward, next_state):
        TD_target = reward + self.discount_factor * max([self.get_Q_value(next_state, (dx, dy)) for dx in [-1, 0, 1] for dy in [-1, 0, 1]])
        TD_error = TD_target - self.get_Q_value(state, action)
        self.Q[(state, action)] = self.get_Q_value(state, action) + self.learning_rate * TD_error



# Main loop
if __name__ == "__main__":
    # Define grid world environment
    width = 5
    height = 5
    goal = (4, 4)
    obstacles = [(1, 1), (2, 2), (3, 3)]
    env = GridWorldPOMDP(width, height, goal, obstacles)

    # Define Q-learning agent
    num_actions = 4  # Up, down, left, right
    agent = QLearningAgentPOMDP(num_actions)

    # Training loop
    num_episodes = 1000
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            # Choose action based on partially observable state
            action = agent.choose_action(state)

            # Take action and observe next state, reward
            next_state, reward, done = env.step(action)

            # Update Q-value based on partially observable state
            agent.update_Q_value(state, action, reward, next_state)

            state = next_state
            total_reward += reward

        print(f"Episode {episode + 1}: Total Reward = {total_reward}")


TypeError: cannot unpack non-iterable int object